In [1]:
import dill

name_to_id = dill.load(open("./../annotation/name_to_id.dill","rb"))
id_to_name = dill.load(open("./../annotation/id_to_name.dill","rb"))

from sklearn.model_selection import StratifiedGroupKFold
import random
import numpy as np
import pyarrow.parquet as pq
import pandas as pd

# Load the data using pyarrow for faster reading
file_path = './../data/GEO/preprocessed/zeroshot_Mvalues.parquet'
table = pq.read_table(file_path)

# Convert to pandas DataFrame if needed
Mv = table.to_pandas().set_index('probe')
Mv.columns = Mv.columns.str.split('_').str[0]
meta = pd.read_csv('./../annotation/zeroshot_meta.csv', header=0, index_col='Sample')

print(Mv.shape, meta.shape)

(299347, 7025) (6631, 9)


In [2]:
training_Mv, training_meta = dill.load(open(f'./../data/GEO/preprocessed/training.dill', 'rb'))
training_meta.shape

(10351, 10)

In [3]:
print(f"Sample overlap: {set(training_meta.index).intersection(set(meta.index))}")

Sample overlap: set()


In [4]:
print(f"Dataset overlap: {set(training_meta.Dataset).intersection(set(meta.Dataset))}")

Dataset overlap: {'GSE95761', 'GSE84743', 'GSE88824', 'GSE205661', 'GSE224807', 'GSE116754', 'GSE82084', 'GSE118260', 'GSE103413', 'GSE118696', 'GSE246935', 'GSE148021', 'GSE48472'}


In [5]:
print(f"Tissue overlap: {set(training_meta['merged.ID']).intersection(set(meta['merged.ID']))}")

Tissue overlap: set()


In [6]:
print("contains fetal: ")
print(meta[meta['Annotated.tissue'].str.contains('fetal', case=False, na=False)])

contains fetal: 
Empty DataFrame
Columns: [UBERON.ID, Annotated.tissue, Dataset, Disease, Normal, Platform, Treatment, idat_check, merged.ID]
Index: []


In [7]:
zeroshot_Mv = Mv.T.loc[meta.index, training_Mv.columns]
zeroshot_meta = meta

print(zeroshot_Mv.shape, zeroshot_meta.shape)

(6631, 297598) (6631, 9)


In [8]:
with open(f'./../data/GEO/preprocessed/zeroshot.dill', 'wb') as f:
    dill.dump([zeroshot_Mv, zeroshot_meta], f)